In [65]:
import pandas
import re

from lab_v2.xlsx_creation import XlsxWorkbook, XlsxSheet

In [66]:
DRAW_T03 = 'cache/draw/draw-T0.3.jsonl'
DRAW_T05 = 'cache/draw/draw-T0.5.jsonl'
DRAW_T07 = 'cache/draw/draw-T0.7.jsonl'

CSQA_T03 = 'cache/csqa/csqa-T0.3.jsonl'
CSQA_T05 = 'cache/csqa/csqa-T0.5.jsonl'
CSQA_T07 = 'cache/csqa/csqa-T0.7.jsonl'

LAST_LETTERS_T03 = 'cache/last_letters/last_letters-T0.3.jsonl'
LAST_LETTERS_T05 = 'cache/last_letters/last_letters-T0.5.jsonl'
LAST_LETTERS_T07 = 'cache/last_letters/last_letters-T0.7.jsonl'

In [67]:
data = pandas.read_json(LAST_LETTERS_T03, lines=True)
data.sort_values(by='id', ascending=False).head(1)

,id,clf,sampler,fit_time,score_time,accuracy,precision-Incorrect,recall-Incorrect,f1-Incorrect,precision-Correct,recall-Correct,f1-Correct
40,"('XGBClassifier(base_score=None, booster=None,...","XGBClassifier(base_score=None, booster=None, c...",RandomUnderSampler(random_state=42),0.103508,0.01508,0.665667,0.8501,0.680538,0.749623,0.394285,0.619001,0.472356


In [68]:
for cache_file_path in [
    'cache/draw/draw-T0.3',
    'cache/draw/draw-T0.5',
    'cache/draw/draw-T0.7',
    'cache/csqa/csqa-T0.3',
    'cache/csqa/csqa-T0.5',
    'cache/csqa/csqa-T0.7',
    'cache/last_letters/last_letters-T0.3',
    'cache/last_letters/last_letters-T0.5',
    'cache/last_letters/last_letters-T0.7',
]:
    xlsx_file_path = 'xlsx/' + '/'.join(cache_file_path.split('/')[1:]) + '.xlsx'
    print(xlsx_file_path)
    data = pandas.read_json(cache_file_path + '.jsonl', lines=True)
    data = data.sort_values(by='id', ascending=False)
    data['name'] = data['clf'].apply(lambda row : re.sub(r'Classifier\(.*', 'Classifier', row, flags=re.DOTALL))
    data['layer_count'] = data['clf'].apply(lambda row : row.count('Dense') - 1)
    data['name'] = data['name'].apply(lambda row : re.sub(r'\(|\)|\'', '', row, flags=re.DOTALL))
    data['name'] = data.apply(lambda row : row['name'] if row['layer_count'] == -1 else 'Tensorflow ' + str(row['layer_count']) + ' layers', axis=1)
    
    data['Average incorrect precision across 5CV'] = data['precision-Incorrect']
    data['Average incorrect recall across 5CV'] = data['recall-Incorrect']
    data['Average correct precision across 5CV'] = data['precision-Correct']
    data['Average correct recall across 5CV'] = data['recall-Correct']

    COLUMNS = ['name', 
    'accuracy', 
    'Average incorrect precision across 5CV', 
    'Average incorrect recall across 5CV', 
    'Average correct precision across 5CV', 
    'Average correct recall across 5CV']
    none_df = data[data['sampler'] == 'None'][COLUMNS]
    undersampled_df = data[data['sampler'] == 'RandomUnderSampler(random_state=42)'][COLUMNS]
    oversampled_df = data[data['sampler'] == 'RandomOverSampler(random_state=42)'][COLUMNS]

    workbook = XlsxWorkbook(xlsx_file_path)
    format = workbook.workbook.add_format()
    format.set_text_wrap()

    for df, name in [
        (none_df, 'None'),
        (undersampled_df, 'Random_Undersampling'),
        (oversampled_df, 'Random_Oversampling')
    ]:
        none: XlsxSheet = workbook.add_sheet(name, dataframe=df)
        workbook.workbook.get_worksheet_by_name(name).set_column(1, 6, 15)
        none_chart = none.add_column_chart('Model Performance', 'H1')
        none_chart.set_size({'width': 720, 'height': 400})
        index = 0
        for _, row in none_df.iterrows():
            index += 1
            none_chart.add_series({
                'name': f'={name}!$B${index + 1}',
                'categories': f'={name}!$D$1:$G$1',
                'values': f'={name}!$D${index + 1}:$G${index + 1}',
            })
    workbook.close()

xlsx/draw/draw-T0.3.xlsx
xlsx/draw/draw-T0.5.xlsx
xlsx/draw/draw-T0.7.xlsx
xlsx/csqa/csqa-T0.3.xlsx
xlsx/csqa/csqa-T0.5.xlsx
xlsx/csqa/csqa-T0.7.xlsx
xlsx/last_letters/last_letters-T0.3.xlsx
xlsx/last_letters/last_letters-T0.5.xlsx
xlsx/last_letters/last_letters-T0.7.xlsx
